# Pandas exercises
All questions are weighted the same in this assignment.
## Part 1

We work with the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table). 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [1]:
import pandas as pd

df = pd.read_csv('../datasets/olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [20]:
def answer_one():
    return df[df['Gold'] == df['Gold'].max()].index.values[0]

answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [12]:
def answer_two():
    diff = (df['Gold']-df['Gold.1'])
    return df[ diff ==diff.max()].index.values[0]

answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [14]:
def answer_three():
    df3 = df[(df['Gold']>0) & (df['Gold.1']>0)]
    ratio = (df3['Gold']-df3['Gold.1'])/(df3['Gold']+df3['Gold.1'])
    return df3[ratio ==ratio.max()].index.values[0]

answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created.

*This function should return a Series named `Points` of length 146*

In [6]:
def answer_four():
    Points = pd.Series(df['Gold.2']*3+df['Silver.2']*2+df['Bronze.2'],index=df.index)
    return Points

answer_four()

Afghanistan                           2
Algeria                              27
Argentina                           130
Armenia                              16
Australasia                          22
                                   ... 
Yugoslavia                          171
Independent Olympic Participants      4
Zambia                                3
Zimbabwe                             18
Mixed team                           38
Length: 146, dtype: int64

## Part 2

For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov/popest/data/counties/totals/2015/CO-EST2015-alldata.html). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](http://www.census.gov/popest/data/counties/totals/2015/files/CO-EST2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it?

*This function should return a single string value.*

In [2]:
import pandas as pd
census_df = pd.read_csv('../datasets/census.csv')

census_df.head(2)

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333


Answer using a simple for-loop:

In [64]:
#%%timeit -n 10
def answer_five():
    df0 = census_df[census_df['SUMLEV']==50]
    states = df0['STNAME'].unique()
    cty = [len(df0[df0['STNAME']==state]) for state in states]
    df0 = pd.DataFrame(cty,index=states,columns=['count'])
    return df0[df0['count']==df0['count'].max()].index.values[0]
    #return df0.idxmax(axis=0).values[0] also works (this is a bit slower)

answer_five()

'Texas'

Answer using a for-loop over a groupby over `STNAME`:

In [63]:
#%%timeit -n 10
def answer_five():
    df0 = census_df[census_df['SUMLEV']==50]
    states = df0['STNAME'].unique()
    cty = []
    for _, frame in df0.groupby('STNAME'):
        cty.append(frame['CTYNAME'].count())

    df0 = pd.DataFrame(cty,index=states)
#    return df0[df0==df0.max()].dropna().index.values[0] 
    return df0[df0.iloc[:,0]==df0.iloc[:,0].max()].index.values[0]

answer_five()

'Texas'

This is by far the fastest method. 

Answer by passing the pandas `size` function through the `.agg` function over a groupby over `STNAME`:

In [55]:
#%%timeit -n 10
def answer_five():
    df0 = census_df[census_df['SUMLEV']==50]
    df0 = df0.groupby('STNAME')['CTYNAME'].agg('size')
#    return df0[df0==df0.max()].index.values[0]

    return df0[df0==df0.max()].index.values[0]

#    df0 = df0.groupby('STNAME').agg({'CTYNAME': 'size'})
#    return df0[df0['CTYNAME']==df0['CTYNAME'].max()].index.values[0]


answer_five()

'Texas'

### Question 6
Only looking at the three most populous counties for each state, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

Answer using a simple for-loop:

In [28]:
#%%timeit -n 10
def answer_six():
    df0 = census_df[census_df['SUMLEV']==50]
    
    df0 = df0.groupby('STNAME')['CENSUS2010POP']
    df0 = df0.apply(lambda x: x.sort_values(ascending=False).iloc[0:3]).groupby('STNAME').sum()

    return list(df0.sort_values(ascending=False).iloc[0:3].index.values)

answer_six()

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [125]:
def answer_seven():
    df0 = census_df[census_df['SUMLEV']==50].copy()
#    df0['COUNTYNAME'] = df0['STNAME'] + ' ' + df0['CTYNAME']
#    df0 = df0[ ['COUNTYNAME'] + list(df0.loc[:,'POPESTIMATE2010':'POPESTIMATE2015']) ]
#    df0.index = df0['COUNTYNAME']
#    df0.drop('COUNTYNAME', axis=1, inplace=True)
    df0.index = df0['CTYNAME']
    df0 = df0.loc[:,'POPESTIMATE2010':'POPESTIMATE2015']
    df0['change'] = df0.max(axis=1) - df0.min(axis=1)
    return df0['change'].sort_values(ascending=False).index.values[0]

answer_seven()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [12]:
def answer_eight():
    df0 = census_df[(census_df['SUMLEV']==50) & ((census_df['REGION']==1) | (census_df['REGION']==2)) & (census_df['POPESTIMATE2015']>census_df['POPESTIMATE2014'])][['STNAME','CTYNAME']]
    return df0[df0['CTYNAME'].str.contains('Washington', na=False)].sort_index(ascending=True)

answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
